In [1]:
from photometry import Spot, ImageData, AperturePhotometry
import numpy as np
import matplotlib.pyplot as plt

### ImageData Module

In [23]:
repo = "embargo_new"
collections = ["LSSTComCam/nightlyValidation"]
instrument='LSSTComCam'
exposure=2024121000100
dataId={"exposure":exposure,"detector":7}
datasetType='postISRCCD'
idata = ImageData(repo=repo)

In [24]:
idata.instrument= instrument
idata.dataId = dataId
idata.datasetType = datasetType
idata.collections = collections
datasets = idata.get_datasets()
# can also access with idata.datasets

One dataset found


In [25]:
image = idata.get_image() #If no dataset passed, it will try to detect one and take it
# can also access with idata.image

One dataset found


In [9]:
#One can also pass a dataset one ref only
idata.dataId = {"exposure":2024121000103,"detector":7}
dataset_2 = idata.get_datasets()
image_2 = idata.get_image(dataset=dataset_2[0])

One dataset found


In [26]:
# We can also use defered datasets
import lsst.daf.butler as daf_butler
butler = daf_butler.Butler(repo, collections=collections)
exposure_handle = daf_butler.DeferredDatasetHandle(butler, datasets[0], None)
idata = ImageData(exposure_handle=exposure_handle, repo=repo)
image = idata.get_image_from_handle()

### Spot Module

In [27]:
sp = Spot(mask_size = 150) #We give the size of the mask pinhole, it remains an indicator only

In [28]:
# Load the image we want

In [29]:
idata = ImageData(exposure_handle, repo=repo)
exposure_handle = daf_butler.DeferredDatasetHandle(butler, datasets[0], None)
image = idata.get_image()

One dataset found


In [30]:
#One can get the estimated area of the spot on the focal plane
sp.get_mask_size()
print(sp.mask_area_fp_px)

46584.59020688224


In [31]:
found_spot = sp.find_spot(idata.img.getImage(), threshold_adu=100,minarea=1000) #Need to pass an afw image*
print(found_spot)

[618 peaks, area=1896, centroid=(3549, 1051.5), 5759 peaks, area=69473, centroid=(1669.8, 2586.53)]


In [32]:
found_spot.sort(key=lambda x: x.getArea(), reverse=True)

In [33]:
iteratively_found_spot = sp.find_spot_iteratively(idata.img.getImage(), threshold_adu_min=10,threshold_adu_max=100000,minarea=1000)
print(iteratively_found_spot)

[4525 peaks, area=43120, centroid=(1666.13, 2592.75)]


In [34]:
sp.get_spot_information() # Get the information from the spot, if no spot passed, will use the last ones detected 
print(sp.centroid, sp.x, sp.y, sp.radius)

(1666.1, 2592.7) 1666.1327226345084 2592.7469851576993 117.15597420637607


In [35]:
#Be carefull here, if multiple spots, only take the first one
sp.get_spot_information(spot = found_spot[0])
print(sp.centroid, sp.x, sp.y, sp.radius)

(1669.8, 2586.5) 1669.7968131504326 2586.5290976350525 148.70757453084389


In [36]:
# We can use the previously determined area
minarea = int(sp.mask_area_fp_px*.9)
found_spot = sp.find_spot(idata.img.getImage(), threshold_adu=100,minarea=minarea)
print(found_spot)
sp.get_spot_information(spot = found_spot[0])
print(sp.centroid, sp.x, sp.y, sp.radius)

[5759 peaks, area=69473, centroid=(1669.8, 2586.53)]
(1669.8, 2586.5) 1669.7968131504326 2586.5290976350525 148.70757453084389


In [37]:
best_spot = sp.find_and_get_best_spot(idata.img.getImage())

In [38]:
best_spot

4525 peaks, area=43120, centroid=(1666.13, 2592.75)

In [39]:
sp.x, sp.y, sp.radius

(1666.1327226345084, 2592.7469851576993, np.float64(117.15597420637607))

In [22]:
sp.x, sp.y, sp.radius

(0.0, 0.0, np.float64(0.5641895835477563))

In [41]:
from astropy.table import Table
a = Table.read("/sdf/group/rubin/user/amouroux/DATA/CBP/Comcamspotmeasurement/test_eo_task/20241210_200.fits")

In [42]:
a

id,coord_ra,coord_dec,parent,detector,exposure,x,y,x_fp,y_fp,radius,signal,bkg_mean,bkg_std,footprint
,rad,rad,,,,,,,,,,,,
int64,float64,float64,int64,bytes10,int64,float32,float32,float32,float32,float32,float32,float32,float32,int32
1,--,--,0,R22_S11,2024121000027,1852.9581,2583.203,-1.8354183,5.8270288,124.18324,28740756.0,5.0715957,0.0,0
2,--,--,0,R22_S00,2024121000027,1920.2115,2274.4946,-43.412884,-39.51005,124.12684,29166896.0,4.763646,0.0,0
3,--,--,0,R22_S02,2024121000027,2153.5437,2646.1648,43.420437,-35.79335,124.19606,29123442.0,5.454216,0.0,0
4,--,--,0,R22_S22,2024121000027,1786.694,2885.7273,39.751938,51.102272,123.55108,27170416.0,6.6865454,0.0,0
5,--,--,0,R22_S10,2024121000027,1727.6956,2395.2678,-45.338043,3.9476776,123.693985,28126378.0,5.137125,0.0,0
6,--,--,0,R22_S01,2024121000027,2040.922,2458.9663,0.044220366,-37.665337,125.01606,30576934.0,5.6451054,0.0,0
7,--,--,0,R22_S12,2024121000027,1973.0867,2768.2979,41.615868,7.6779795,124.08836,28464264.0,5.027025,0.0,0
8,--,--,0,R22_S20,2024121000027,1548.7593,2520.468,-47.127407,47.44968,123.72357,28135420.0,4.4913683,0.0,0


#### Known spots

In [25]:
x, y, radius = sp.x, sp.y, sp.radius #Using previously found spot

In [26]:
sp = Spot(x=x ,y=y , radius=radius) #We give the size of the mask pinhole, it remains an indicator only

### AperturePhotometry module

It is mandatory to use a spot and an image

In [2]:
sp = Spot(x=1670 ,y=2587 , radius=149)
dataId={"exposure":2024121000100,"detector":7}
idata = ImageData("embargo_new", "LSSTComCam/nightlyValidation", "LSSTComCam", dataId, "postISRCCD")
idata.get_image()
ap = AperturePhotometry(image=idata, spot=sp)

One dataset found


In [3]:
background_2d_thrshld = ap.get_2d_background_threshold()

In [4]:
bkg_aperture = ap.generate_aperture(centroid=ap.Spot.centroid, radius=400)
background_2d_ap = ap.get_2d_background_aperture(aperture = bkg_aperture)

In [5]:
mean_bkg = ap.get_mean_background()
print(mean_bkg)

5.139276


In [6]:
dark_exp_bkg = ap.get_dark_background()

One dataset found


In [28]:
substracted_background_image = ap.get_substracted_background_image(background = mean_bkg)

In [29]:
aperture = ap.generate_aperture(ap.Spot.centroid, radius=200)
photometry = ap.do_aperture_photometry(image=substracted_background_image, aperture=aperture)
print(photometry)

72940408.17241915


In [14]:
forced_photometry = ap.do_forced_aperture_photometry(centroid=ap.Spot.centroid, radius=200)
print(photometry)

72968438.49451017


In [21]:
from astropy.table import Table 

In [22]:
a = Table.read("/sdf/group/rubin/user/amouroux/DATA/CBP/Comcamspotmeasurement/test_eo_task/20241210_200.fits")

In [23]:
a

id,coord_ra,coord_dec,parent,detector,exposure,x,y,x_fp,y_fp,radius,signal,bkg_mean,bkg_std,footprint
,rad,rad,,,,,,,,,,,,
int64,float64,float64,int64,bytes10,int64,float32,float32,float32,float32,float32,float32,float32,float32,int32
1,--,--,0,R22_S11,0,1853.0692,2581.1003,-1.8343077,5.8060026,122.516205,26145666.0,3.4761868,0.0,0
2,--,--,0,R22_S10,0,1727.8605,2393.0173,-45.336395,3.9251745,121.77088,25607020.0,3.2762313,0.0,0
3,--,--,0,R22_S02,0,2153.1675,2644.2354,43.416676,-35.812645,122.37973,26495380.0,4.0176535,0.0,0
4,--,--,0,R22_S22,0,1786.7782,2883.3352,39.75278,51.078354,121.59954,24664092.0,3.9083252,0.0,0
5,--,--,0,R22_S01,0,2040.9208,2457.2803,0.044207193,-37.682198,123.30737,27809480.0,4.019445,0.0,0
